#### The code below is adapted from the book "Programming Bitcoin" written by Jimmy Song

In [101]:
class FieldElement:
    def __init__(self, num, prime):
        if num >= prime or num < 0:
            raise ValueError("Number {0} is not in the field range 0 to {1}".format(num, prime - 1))
        self.num = num
        self.prime = prime
        
    def __eq__(self, other):
        if not other:
            return False
        elif self.num == other.num and self.prime == other.prime:
            return True
        else:
            return False
    
    def __ne__(self, other):
        if not other:
            return False
        elif self.num != other.num or self.prime != other.prime:
            return True
        else:
            return False
        
    def __repr__(self):
        return 'FieldElement_{0}({1})'.format(self.prime, self.num)
    
    def __add__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot add two numbers in different fields.")
        num = (self.num + other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __sub__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot subtract two numbers in different fields.")
        num = (self.num - other.num) % self.prime
        return self.__class__(num, self.prime)
    
    def __mul__(self, other):
        if self.prime != other.prime:
            raise TypeError("Cannot multiply two numbers in different fields")
        num = 0
        for _ in range(other.num):
            num += self.num
        num = num % self.prime
        return self.__class__(num, self.prime)
    
    def __pow__(self, exponent):
        num = (self.num ** exponent) % self.prime
        return self.__class__(num, self.prime)

In [102]:
a = FieldElement(12, 13)
b = FieldElement(5, 13)
c = FieldElement(4, 13)
d = FieldElement(7, 13)

In [103]:
a
a == b
a != b

True

In [104]:
a - b == d
a + b == c

True

In [105]:
a * b

FieldElement_13(8)

In [107]:
a ** 3

FieldElement_13(12)

In [110]:
a = FieldElement(3, 13)
b = FieldElement(1, 13)
a ** 3 == b

True